# Standardization and Normalization

**Essential Data Scaling Techniques for PCA and Machine Learning**

---

## Introduction

In the previous notebooks, we learned about variance, covariance, and correlation. But we encountered an important question: **How do we compare variables measured in different units?**

Consider this agricultural scenario:
- **Soil pH**: Range 5.0 to 8.0 (unitless, small scale)
- **Nitrogen content**: Range 10 to 200 ppm (large scale)
- **Organic matter**: Range 1% to 6% (percentage)

If we analyze these together without scaling, variables with larger ranges will **dominate** the analysis. This is especially problematic for **Principal Component Analysis (PCA)**, where variance matters!

### Why This Matters for PCA ⭐⭐

**PCA finds directions of maximum variance.** If one variable has a much larger variance simply because of its units (not because it's more important), PCA will focus on that variable and miss important patterns in other variables.

**The solution**: Standardization and Normalization make variables comparable by putting them on the same scale.

### Learning Objectives

By the end of this notebook, you will:

1. ✓ Understand **why** scaling is necessary for PCA
2. ✓ Master **z-score standardization** (most common for PCA)
3. ✓ Learn **min-max normalization** and when to use it
4. ✓ Understand **robust scaling** for data with outliers
5. ✓ Visualize the **effect of scaling** on data
6. ✓ Know **when to standardize** before PCA
7. ✓ See how scaling affects **covariance vs correlation** matrices

**Agricultural Context**: We'll use multi-variable soil data (pH, nitrogen, phosphorus, moisture) to demonstrate why scaling matters for analyzing soil properties together.

---

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Configure plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("✓ Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## 1. The Scaling Problem

Let's start by seeing the problem firsthand with real agricultural data.

### Agricultural Scenario

You have soil samples from 50 agricultural fields with the following measurements:
- **pH**: 5.5 to 7.5 (small range, unitless)
- **Nitrogen (N)**: 20 to 180 ppm (large range)
- **Phosphorus (P)**: 10 to 90 ppm (medium range)
- **Soil Moisture**: 15% to 35% (medium range)

You want to understand the overall soil quality patterns. Should you analyze these variables directly, or scale them first?

In [ ]:
# Generate realistic agricultural soil data
n_samples = 50

# Different variables with different scales
soil_pH = np.random.normal(6.5, 0.5, n_samples)  # Small range: ~5.5 to 7.5
nitrogen_ppm = np.random.normal(100, 40, n_samples)  # Large range: ~20 to 180
phosphorus_ppm = np.random.normal(50, 20, n_samples)  # Medium range: ~10 to 90
moisture_pct = np.random.normal(25, 5, n_samples)  # Medium range: ~15 to 35

# Create DataFrame
soil_data = pd.DataFrame({
    'pH': soil_pH,
    'Nitrogen_ppm': nitrogen_ppm,
    'Phosphorus_ppm': phosphorus_ppm,
    'Moisture_%': moisture_pct
})

print("🌾 Soil Property Data Summary")
print("=" * 60)
print(soil_data.describe().round(2))
print("\n📊 Notice the very different scales!")
print(f"  • pH range: {soil_data['pH'].min():.2f} to {soil_data['pH'].max():.2f}")
print(f"  • Nitrogen range: {soil_data['Nitrogen_ppm'].min():.2f} to {soil_data['Nitrogen_ppm'].max():.2f}")
print(f"  • Phosphorus range: {soil_data['Phosphorus_ppm'].min():.2f} to {soil_data['Phosphorus_ppm'].max():.2f}")
print(f"  • Moisture range: {soil_data['Moisture_%'].min():.2f} to {soil_data['Moisture_%'].max():.2f}")

In [ ]:
# Visualize the scaling problem
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('🌾 Soil Properties: Different Scales Create Comparison Problems', 
             fontsize=16, fontweight='bold', y=1.00)

variables = ['pH', 'Nitrogen_ppm', 'Phosphorus_ppm', 'Moisture_%']
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A']

for idx, (var, color, ax) in enumerate(zip(variables, colors, axes.flat)):
    ax.hist(soil_data[var], bins=15, color=color, alpha=0.7, edgecolor='black')
    ax.axvline(soil_data[var].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
    ax.set_xlabel(var, fontsize=12, fontweight='bold')
    ax.set_ylabel('Frequency', fontsize=11)
    ax.set_title(f'{var}\nRange: {soil_data[var].min():.1f} to {soil_data[var].max():.1f}', 
                 fontsize=12)
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n⚠️  THE PROBLEM: Variables have vastly different ranges!")
print("   → Nitrogen varies from 20-180 (range ~160)")
print("   → pH varies from 5.5-7.5 (range ~2)")
print("   → Without scaling, nitrogen will DOMINATE any analysis!")

### The Variance Problem for PCA

Let's calculate the variance of each variable to see the problem clearly:

In [ ]:
# Calculate variance for each variable
variances = soil_data.var()

print("📊 Variance of Each Soil Property (Unscaled)")
print("=" * 60)
for var, variance in variances.items():
    print(f"  {var:20s}: {variance:10.2f}")

print("\n⚡ KEY INSIGHT FOR PCA:")
print(f"  Nitrogen variance is {variances['Nitrogen_ppm'] / variances['pH']:.0f}x larger than pH variance!")
print("  → PCA will focus almost entirely on Nitrogen")
print("  → pH patterns will be ignored, even if they're important!")
print("\n💡 SOLUTION: Standardize the data so all variables contribute equally!")

In [ ]:
# Visualize variance comparison
fig, ax = plt.subplots(figsize=(12, 6))

bars = ax.bar(variances.index, variances.values, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
ax.set_ylabel('Variance', fontsize=13, fontweight='bold')
ax.set_title('🚨 The Scaling Problem: Variance Dominance\n(Nitrogen has 640x more variance than pH!)', 
             fontsize=14, fontweight='bold')
ax.grid(True, axis='y', alpha=0.3)

# Add value labels on bars
for bar, var in zip(bars, variances.values):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{var:.1f}',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.xticks(rotation=15, ha='right')
plt.tight_layout()
plt.show()

print("\n🎯 Without standardization, PCA would be dominated by Nitrogen!")

---

## 2. Z-Score Standardization ⭐⭐

**The most common scaling method for PCA!**

### Definition

Z-score standardization transforms each value by:

$$z = \frac{x - \mu}{\sigma}$$

Where:
- $x$ = original value
- $\mu$ = mean of the variable
- $\sigma$ = standard deviation of the variable
- $z$ = standardized value (z-score)

### Properties After Standardization

- **Mean = 0**: Data is centered at zero
- **Standard Deviation = 1**: All variables have the same spread
- **Shape Preserved**: The distribution shape doesn't change
- **Units Removed**: Z-scores are unitless

### Interpretation

A z-score tells you **how many standard deviations** a value is from the mean:
- $z = 0$: At the mean
- $z = 1$: One standard deviation above the mean
- $z = -2$: Two standard deviations below the mean

### Agricultural Example

Let's standardize the soil pH data step by step:

In [ ]:
# Manual z-score standardization for pH
pH_values = soil_data['pH'].values

# Step 1: Calculate mean
pH_mean = np.mean(pH_values)
print("Step 1: Calculate Mean")
print(f"  μ = {pH_mean:.3f}")

# Step 2: Calculate standard deviation
pH_std = np.std(pH_values, ddof=1)  # Sample SD
print("\nStep 2: Calculate Standard Deviation")
print(f"  σ = {pH_std:.3f}")

# Step 3: Subtract mean from each value
pH_centered = pH_values - pH_mean
print("\nStep 3: Center the Data (x - μ)")
print(f"  First 5 centered values: {pH_centered[:5].round(3)}")

# Step 4: Divide by standard deviation
pH_standardized_manual = pH_centered / pH_std
print("\nStep 4: Scale by SD → z = (x - μ) / σ")
print(f"  First 5 z-scores: {pH_standardized_manual[:5].round(3)}")

# Verify properties
print("\n✓ Verification:")
print(f"  Mean of z-scores: {np.mean(pH_standardized_manual):.10f}  (should be ≈ 0)")
print(f"  SD of z-scores: {np.std(pH_standardized_manual, ddof=1):.10f}  (should be ≈ 1)")

# Compare with NumPy's implementation
pH_standardized_numpy = (pH_values - pH_mean) / pH_std
print("\n✓ NumPy matches our manual calculation:", 
      np.allclose(pH_standardized_manual, pH_standardized_numpy))

### Standardize All Soil Variables

In [ ]:
# Standardize all variables
soil_standardized = pd.DataFrame()

for column in soil_data.columns:
    values = soil_data[column].values
    mean_val = np.mean(values)
    std_val = np.std(values, ddof=1)
    
    # Z-score standardization
    z_scores = (values - mean_val) / std_val
    
    soil_standardized[column + '_z'] = z_scores

print("🌾 Standardized Soil Data Summary")
print("=" * 60)
print(soil_standardized.describe().round(3))

print("\n✓ Notice:")
print("  • All means are now ≈ 0")
print("  • All standard deviations are now ≈ 1")
print("  • All variables are now on the SAME SCALE!")

In [ ]:
# Visualize before and after standardization
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle('🔄 Before and After Standardization\n(Same Distribution Shape, Different Scale)', 
             fontsize=16, fontweight='bold', y=0.98)

# Original data
for idx, (var, color) in enumerate(zip(soil_data.columns, colors)):
    ax = axes[0, idx]
    ax.hist(soil_data[var], bins=15, color=color, alpha=0.7, edgecolor='black')
    ax.axvline(soil_data[var].mean(), color='red', linestyle='--', linewidth=2)
    ax.set_title(f'Original: {var}', fontsize=11, fontweight='bold')
    ax.set_ylabel('Frequency', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    # Add mean and SD text
    ax.text(0.05, 0.95, f'μ = {soil_data[var].mean():.1f}\nσ = {soil_data[var].std():.1f}',
            transform=ax.transAxes, fontsize=9, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Standardized data
for idx, (var, color) in enumerate(zip(soil_standardized.columns, colors)):
    ax = axes[1, idx]
    ax.hist(soil_standardized[var], bins=15, color=color, alpha=0.7, edgecolor='black')
    ax.axvline(soil_standardized[var].mean(), color='red', linestyle='--', linewidth=2)
    ax.set_title(f'Standardized: {var}', fontsize=11, fontweight='bold')
    ax.set_xlabel('Z-score', fontsize=10)
    ax.set_ylabel('Frequency', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    # Add mean and SD text
    ax.text(0.05, 0.95, f'μ = {soil_standardized[var].mean():.3f}\nσ = {soil_standardized[var].std():.3f}',
            transform=ax.transAxes, fontsize=9, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

print("\n🎯 KEY OBSERVATION:")
print("  • Distribution SHAPE is preserved")
print("  • But now all variables are on the SAME SCALE (-3 to +3)")
print("  • Perfect for PCA!")

### Variance After Standardization

In [ ]:
# Compare variances before and after
variances_standardized = soil_standardized.var()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('⚖️ Variance Comparison: Before and After Standardization', 
             fontsize=14, fontweight='bold')

# Before
ax1.bar(variances.index, variances.values, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
ax1.set_ylabel('Variance', fontsize=12, fontweight='bold')
ax1.set_title('BEFORE Standardization\n(Nitrogen dominates!)', fontsize=12)
ax1.grid(True, axis='y', alpha=0.3)
ax1.tick_params(axis='x', rotation=15)

for i, (var, v) in enumerate(zip(variances.index, variances.values)):
    ax1.text(i, v, f'{v:.1f}', ha='center', va='bottom', fontweight='bold')

# After
ax2.bar(range(len(variances_standardized)), variances_standardized.values, 
        color=colors, alpha=0.7, edgecolor='black', linewidth=2)
ax2.set_ylabel('Variance', fontsize=12, fontweight='bold')
ax2.set_title('AFTER Standardization\n(All equal to 1!)', fontsize=12)
ax2.set_xticks(range(len(variances_standardized)))
ax2.set_xticklabels(soil_data.columns, rotation=15, ha='right')
ax2.grid(True, axis='y', alpha=0.3)
ax2.set_ylim([0, 1.2])

for i, v in enumerate(variances_standardized.values):
    ax2.text(i, v, f'{v:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n🎯 CRITICAL FOR PCA:")
print("  • Before: Variances ranged from 0.25 to 1603")
print("  • After: All variances are exactly 1.0")
print("  • Now PCA will treat all variables EQUALLY!")

---

## 3. Min-Max Normalization

Another common scaling technique that scales data to a specific range (usually 0 to 1).

### Formula

$$x_{\text{normalized}} = \frac{x - x_{\min}}{x_{\max} - x_{\min}}$$

### Properties

- **Range**: Always between 0 and 1
- **Preserves**: Original distribution shape
- **Use When**: You need a specific range (e.g., for neural networks)
- **Sensitive to**: Outliers (they compress the range)

### When to Use Normalization vs Standardization

- **Standardization (Z-score)**: Use for PCA, when you don't know the distribution, or when you have outliers that shouldn't dominate
- **Normalization (Min-Max)**: Use when you need a bounded range, for neural networks, or when all values must be positive

In [ ]:
# Min-max normalization for pH
pH_values = soil_data['pH'].values

# Step 1: Find min and max
pH_min = np.min(pH_values)
pH_max = np.max(pH_values)
print("Step 1: Find Min and Max")
print(f"  Min = {pH_min:.3f}")
print(f"  Max = {pH_max:.3f}")
print(f"  Range = {pH_max - pH_min:.3f}")

# Step 2: Apply formula
pH_normalized = (pH_values - pH_min) / (pH_max - pH_min)
print("\nStep 2: Apply Formula → (x - min) / (max - min)")
print(f"  First 5 normalized values: {pH_normalized[:5].round(3)}")

# Verify properties
print("\n✓ Verification:")
print(f"  Min of normalized data: {np.min(pH_normalized):.10f}  (should be 0)")
print(f"  Max of normalized data: {np.max(pH_normalized):.10f}  (should be 1)")
print(f"  Mean of normalized data: {np.mean(pH_normalized):.3f}  (not necessarily 0.5)")

In [ ]:
# Normalize all variables
soil_normalized = pd.DataFrame()

for column in soil_data.columns:
    values = soil_data[column].values
    min_val = np.min(values)
    max_val = np.max(values)
    
    # Min-max normalization
    normalized = (values - min_val) / (max_val - min_val)
    
    soil_normalized[column + '_norm'] = normalized

print("🌾 Normalized Soil Data Summary")
print("=" * 60)
print(soil_normalized.describe().round(3))

print("\n✓ Notice:")
print("  • All minimums are now 0.0")
print("  • All maximums are now 1.0")
print("  • Means are NOT necessarily 0.5 (depends on distribution)")

In [ ]:
# Compare standardization vs normalization
fig, axes = plt.subplots(3, 4, figsize=(16, 10))
fig.suptitle('📊 Comparison: Original vs Standardized vs Normalized', 
             fontsize=16, fontweight='bold', y=0.995)

for idx, (var, color) in enumerate(zip(soil_data.columns, colors)):
    # Original
    axes[0, idx].hist(soil_data[var], bins=15, color=color, alpha=0.7, edgecolor='black')
    axes[0, idx].set_title(f'Original: {var}', fontsize=10, fontweight='bold')
    axes[0, idx].set_ylabel('Frequency', fontsize=9)
    axes[0, idx].grid(True, alpha=0.3)
    
    # Standardized
    axes[1, idx].hist(soil_standardized[var + '_z'], bins=15, color=color, alpha=0.7, edgecolor='black')
    axes[1, idx].set_title(f'Standardized (Z-score)', fontsize=10, fontweight='bold')
    axes[1, idx].set_ylabel('Frequency', fontsize=9)
    axes[1, idx].set_xlabel('Z-score', fontsize=9)
    axes[1, idx].grid(True, alpha=0.3)
    axes[1, idx].axvline(0, color='red', linestyle='--', linewidth=1.5)
    
    # Normalized
    axes[2, idx].hist(soil_normalized[var + '_norm'], bins=15, color=color, alpha=0.7, edgecolor='black')
    axes[2, idx].set_title(f'Normalized (0-1)', fontsize=10, fontweight='bold')
    axes[2, idx].set_ylabel('Frequency', fontsize=9)
    axes[2, idx].set_xlabel('Normalized Value', fontsize=9)
    axes[2, idx].grid(True, alpha=0.3)
    axes[2, idx].set_xlim([-0.1, 1.1])

plt.tight_layout()
plt.show()

print("\n🔍 Key Differences:")
print("  Standardization (Z-score):")
print("    ✓ Mean = 0, SD = 1")
print("    ✓ Range: typically -3 to +3 (can be beyond)")
print("    ✓ Best for PCA")
print("\n  Normalization (Min-Max):")
print("    ✓ Range = exactly 0 to 1")
print("    ✓ Mean = varies by distribution")
print("    ✓ Best for neural networks, bounded ranges")

---

## 4. Robust Scaling

When data has **outliers**, robust scaling uses the **median** and **interquartile range (IQR)** instead of mean and standard deviation.

### Formula

$$x_{\text{robust}} = \frac{x - \text{median}}{\text{IQR}}$$

Where IQR = Q3 - Q1

### Properties

- **Robust to outliers**: Median and IQR are not affected by extreme values
- **Median = 0** after scaling
- **Range**: Not bounded to [0, 1] or [-3, 3]
- **Use When**: Your data has significant outliers that you want to keep

### Agricultural Example with Outliers

In [ ]:
# Create data with outliers
yield_data = np.random.normal(5000, 800, 48)  # Normal yields
outliers = np.array([1500, 9500])  # Two outlier fields
yield_with_outliers = np.concatenate([yield_data, outliers])

print("🌾 Wheat Yield Data (kg/hectare)")
print("=" * 60)
print(f"Mean: {np.mean(yield_with_outliers):.1f}")
print(f"Median: {np.median(yield_with_outliers):.1f}")
print(f"Standard Deviation: {np.std(yield_with_outliers, ddof=1):.1f}")
print(f"IQR: {np.percentile(yield_with_outliers, 75) - np.percentile(yield_with_outliers, 25):.1f}")
print(f"\nMin: {np.min(yield_with_outliers):.1f} (OUTLIER!)")
print(f"Max: {np.max(yield_with_outliers):.1f} (OUTLIER!)")

# Standard scaling
mean_yield = np.mean(yield_with_outliers)
std_yield = np.std(yield_with_outliers, ddof=1)
yield_standardized = (yield_with_outliers - mean_yield) / std_yield

# Robust scaling
median_yield = np.median(yield_with_outliers)
q1 = np.percentile(yield_with_outliers, 25)
q3 = np.percentile(yield_with_outliers, 75)
iqr_yield = q3 - q1
yield_robust = (yield_with_outliers - median_yield) / iqr_yield

print("\n📊 Scaling Comparison:")
print(f"Standard scaling range: {yield_standardized.min():.2f} to {yield_standardized.max():.2f}")
print(f"Robust scaling range: {yield_robust.min():.2f} to {yield_robust.max():.2f}")
print("\n💡 Robust scaling compresses outliers less!")

In [ ]:
# Visualize the difference
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('🎯 Robust Scaling vs Standard Scaling (With Outliers)', 
             fontsize=14, fontweight='bold')

# Original data
axes[0].hist(yield_with_outliers, bins=20, color='#FF6B6B', alpha=0.7, edgecolor='black')
axes[0].axvline(mean_yield, color='red', linestyle='--', linewidth=2, label='Mean')
axes[0].axvline(median_yield, color='blue', linestyle='--', linewidth=2, label='Median')
axes[0].set_xlabel('Yield (kg/ha)', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Original Data\n(with outliers)', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Standard scaling
axes[1].hist(yield_standardized, bins=20, color='#4ECDC4', alpha=0.7, edgecolor='black')
axes[1].axvline(0, color='red', linestyle='--', linewidth=2)
axes[1].set_xlabel('Z-score', fontsize=11)
axes[1].set_ylabel('Frequency', fontsize=11)
axes[1].set_title('Standard Scaling\n(outliers affect all values)', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Robust scaling
axes[2].hist(yield_robust, bins=20, color='#45B7D1', alpha=0.7, edgecolor='black')
axes[2].axvline(0, color='blue', linestyle='--', linewidth=2)
axes[2].set_xlabel('Robust Score', fontsize=11)
axes[2].set_ylabel('Frequency', fontsize=11)
axes[2].set_title('Robust Scaling\n(outliers isolated)', fontsize=12, fontweight='bold')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🎯 When to Use Robust Scaling:")
print("  ✓ Data has significant outliers")
print("  ✓ Outliers are real (not errors) and should be kept")
print("  ✓ You want outliers to have less influence on scaling")
print("\n⚠️  For PCA: Usually use standard z-score scaling")
print("   But if outliers are problematic, robust scaling can help!")

---

## 5. Impact on Covariance vs Correlation ⭐

**This is CRITICAL for understanding PCA!**

Remember from the previous notebook:
- **Covariance Matrix**: Uses unscaled data
- **Correlation Matrix**: Uses standardized data

PCA can use either matrix, but:
- **Covariance-based PCA**: Variables with larger variance dominate
- **Correlation-based PCA**: All variables contribute equally (equivalent to standardizing first)

### Let's See the Difference

In [ ]:
# Compute covariance matrix (unscaled data)
cov_matrix = soil_data.cov()

# Compute correlation matrix (equivalent to standardized covariance)
corr_matrix = soil_data.corr()

# Also compute covariance of standardized data
cov_matrix_standardized = soil_standardized.cov()

print("📊 Covariance Matrix (Unscaled Data)")
print("=" * 60)
print(cov_matrix.round(2))
print("\n⚠️  Notice: Diagonal values (variances) are VERY different!")
print(f"   pH variance: {cov_matrix.iloc[0,0]:.2f}")
print(f"   Nitrogen variance: {cov_matrix.iloc[1,1]:.2f}")
print(f"   → Nitrogen is {cov_matrix.iloc[1,1] / cov_matrix.iloc[0,0]:.0f}x more variable")

print("\n" + "=" * 60)
print("📊 Correlation Matrix (Standardized Covariance)")
print("=" * 60)
print(corr_matrix.round(3))
print("\n✓ Notice: Diagonal values are all 1.0 (all variables equal)")

print("\n" + "=" * 60)
print("📊 Covariance Matrix of Standardized Data")
print("=" * 60)
print(cov_matrix_standardized.round(3))
print("\n✓ This equals the correlation matrix!")
print("   Covariance of standardized data = Correlation")

In [ ]:
# Visualize the matrices side by side
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
fig.suptitle('🔍 Covariance vs Correlation Matrix for PCA', 
             fontsize=14, fontweight='bold')

# Covariance matrix heatmap
sns.heatmap(cov_matrix, annot=True, fmt='.1f', cmap='RdYlBu_r', 
            cbar_kws={'label': 'Covariance'}, ax=axes[0],
            linewidths=1, linecolor='black')
axes[0].set_title('Covariance Matrix\n(Unscaled - Nitrogen Dominates)', 
                  fontsize=12, fontweight='bold')

# Correlation matrix heatmap
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='RdYlBu_r', 
            cbar_kws={'label': 'Correlation'}, ax=axes[1],
            linewidths=1, linecolor='black', vmin=-1, vmax=1)
axes[1].set_title('Correlation Matrix\n(Standardized - All Equal)', 
                  fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n🎯 FOR PCA:")
print("  Left (Covariance): PCA will focus mainly on Nitrogen")
print("  Right (Correlation): PCA will consider all variables equally")
print("\n💡 RULE OF THUMB:")
print("  • Different units or scales → STANDARDIZE → Use correlation")
print("  • Same units and scale → Can use covariance")
print("  • When in doubt → STANDARDIZE!")

---

## 6. When to Standardize for PCA ⭐⭐

### Decision Framework

**ALWAYS Standardize When:**
1. ✓ Variables have **different units** (pH, ppm, %, kg, etc.)
2. ✓ Variables have **different scales** (0-10 vs 0-1000)
3. ✓ You want **all variables to contribute equally**
4. ✓ You're **unsure** (standardizing is safer!)

**Can Skip Standardization When:**
1. ✓ All variables have the **same units**
2. ✓ Variables are on **similar scales**
3. ✓ Larger variance variables are **truly more important**
4. ✓ You **specifically want** variables with more variance to dominate

### Agricultural Examples

In [ ]:
print("🌾 Agricultural PCA Scenarios")
print("=" * 70)

scenarios = [
    {
        'name': 'Soil Chemistry Analysis',
        'variables': ['pH (5-8)', 'N (ppm)', 'P (ppm)', 'K (ppm)', 'Moisture (%)'],
        'standardize': 'YES ✓',
        'reason': 'Different units and scales - must standardize!'
    },
    {
        'name': 'Multi-Nutrient Analysis',
        'variables': ['N (ppm)', 'P (ppm)', 'K (ppm)', 'Ca (ppm)', 'Mg (ppm)'],
        'standardize': 'OPTIONAL',
        'reason': 'Same units, but if nutrient ranges differ, standardize'
    },
    {
        'name': 'Crop Yield Analysis',
        'variables': ['Wheat (kg/ha)', 'Corn (kg/ha)', 'Soy (kg/ha)'],
        'standardize': 'DEPENDS',
        'reason': 'Same units, but different crops have different yield ranges'
    },
    {
        'name': 'Weather Patterns',
        'variables': ['Temp (°C)', 'Rainfall (mm)', 'Humidity (%)', 'Wind (km/h)'],
        'standardize': 'YES ✓',
        'reason': 'Different units and scales - must standardize!'
    }
]

for i, scenario in enumerate(scenarios, 1):
    print(f"\n{i}. {scenario['name']}")
    print(f"   Variables: {', '.join(scenario['variables'])}")
    print(f"   Standardize? {scenario['standardize']}")
    print(f"   Reason: {scenario['reason']}")

print("\n" + "=" * 70)
print("\n💡 GOLDEN RULE: When in doubt, STANDARDIZE!")
print("   It's safer and ensures all variables contribute to PCA.")

---

## 7. Summary and Key Insights

### What We Learned

1. **The Problem**: Variables with different scales have different variances, causing some to dominate PCA

2. **Z-Score Standardization** ⭐⭐ (Most Important for PCA)
   - Formula: $z = (x - \mu) / \sigma$
   - Properties: Mean = 0, SD = 1
   - Use for: PCA, most ML algorithms

3. **Min-Max Normalization**
   - Formula: $(x - \text{min}) / (\text{max} - \text{min})$
   - Properties: Range = 0 to 1
   - Use for: Neural networks, bounded ranges

4. **Robust Scaling**
   - Formula: $(x - \text{median}) / \text{IQR}$
   - Properties: Resistant to outliers
   - Use for: Data with significant outliers

5. **Impact on PCA**
   - Covariance matrix PCA: Variables with larger variance dominate
   - Correlation matrix PCA: All variables contribute equally
   - Standardizing data → PCA uses correlation matrix

### Connection to PCA ⭐⭐

```python
# WITHOUT standardization - WRONG for mixed units!
cov_matrix = np.cov(soil_data.T)  # Nitrogen dominates
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

# WITH standardization - CORRECT!
soil_standardized = (soil_data - soil_data.mean()) / soil_data.std()
cov_matrix_std = np.cov(soil_standardized.T)  # All variables equal
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix_std)
```

### Decision Framework for PCA

```
Are variables in different units? ────YES────> STANDARDIZE!
         │
        NO
         │
         ▼
Are variables on very different scales? ──YES──> STANDARDIZE!
         │
        NO
         │
         ▼
Do you want all variables to contribute equally? ──YES──> STANDARDIZE!
         │
        NO
         │
         ▼
   Can skip standardization (rare!)
```

### Agricultural Takeaways

- 🌾 Soil analysis: Different nutrients, pH, moisture → **Always standardize**
- 🌱 Crop yields: Different crops with different ranges → **Standardize**
- ☀️ Weather data: Temperature, rainfall, humidity → **Always standardize**
- 🚜 Management practices: Different units and scales → **Always standardize**

### Next Steps

In the next notebook, we'll explore **data distributions** (skewness, kurtosis, normality) and understand how the shape of our data affects PCA and other analyses.

You now have the essential preprocessing knowledge for PCA! In future notebooks, we'll:
1. Explore data distributions
2. Detect and handle outliers
3. Apply these concepts to real agricultural datasets
4. **Finally perform PCA** with properly prepared data!

---

**Remember**: Standardization is one of the most important preprocessing steps for PCA. When in doubt, standardize! 📊✨